# Demo of pig detection and tracking models
This Colab notebook demonstrates the capabilities of the pig detection and tracking models presented in the paper "Benchmarking pig detection and tracking under diverse and challenging conditions".

# Create environment

First, we need to create a suitable environment to run the models. The entire installation process takes roughly 4 minutes. **Do not get worried by red warning messages and also not by the big window that pops up and tells you to restart the session.** Just wait for 4 minutes until the installation is over. **After it is finished**, restart the session by clicking on the pop up window. You can then continue to run the next cells.

In [ ]:
!pip install torch==2.0.0
!pip install torchvision==0.15.1
!pip install -U openmim
!mim install mmengine
!mim install "mmcv==2.0.0"
!mim install mmdet==3.3.0
!mim install mmyolo==0.6.0
!pip install numpy==1.26.4
!pip install munch
!pip install loguru
!pip install ftfy
!pip install lap
!pip install filterpy


# Clone repo and load packages
After the environment has been set up, we need to clone the PigBench repository and load the necessary packages.

In [ ]:
!git clone https://github.com/jonaden94/PigBench

In [ ]:
%cd PigBench/detection/tools/inference
import mmcv
import os
import sys
from mmdet.utils import register_all_modules
from mmdet.apis import init_detector, inference_detector
from visualization_utils import draw_bboxes
sys.path.append('../../')

# detection demo

Before running the below code cells, please download the pre-trained Co-DINO checkpoint (named codino_swin.pth) from this data repository: https://data.goettingen-research-online.de/dataset.xhtml?persistentId=doi:10.25625/I6UYE9

After you did so, open your google drive and navigate to the ``MyDrive`` folder. Depending on your language settings, the folder might appear in a translated form (e.g. ``Meine Ablage`` in German). Within the MyDrive folder, create a folder named ``PigDetect`` and put the downloaded model weights there. The full path in google drive to the model weights should be ``MyDrive/PigDetect/co_dino_swin.pth``. You can then connect google drive with Colab by running the following code block:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

If you successfully mounted google drive, you can now run the below code to load the model.

In [ ]:
## config and checkpoint file paths
config_path = '../../configs/co_detr/co_dino_swin.py'
checkpoint_path = '../../../../drive/MyDrive/PigDetect/codino_swin.pth'

register_all_modules(init_default_scope=False)
model = init_detector(config_path, checkpoint_path, device='cuda:0')

The next block runs inference on an example image. Feel free to put other pig images in your google drive to run inference on them!

In [ ]:
# run inference on a single example image
image_path = '../../data/example_images/demo_image1.jpg'

# Alternatively, put your own images in google drive in the PigDetect folder and adapt the below path!
# image_path = '../../../../drive/MyDrive/PigDetect/name_of_your_image.jpg'

image = mmcv.imread(image_path, channel_order='rgb')
result = inference_detector(model, image) # you can also provide image_path1 as input

scores0 = result.pred_instances.scores.cpu().numpy()
bboxes0 = result.pred_instances.bboxes.cpu().numpy()

The next block visualizes the predicted bounding boxes.

In [ ]:
# score_thresh is the minimum score of a bbox that is required for it to be visualized
draw_bboxes(image_path=image_path, bboxes=bboxes0, scores=scores0,
            score_thresh=0.4, save_path=None, linewidth=2, show_scores=True, figsize=(15, 15), score_fontsize=6)

# tracking demo

Before running the below code cell, please download the pre-trained Co-DINO checkpoint as described above in the detection demo. If you already did so, that's great! You also need to put a video in google drive to run inference. As an example, you can download the ``PigTrackVideos.zip`` file from https://data.goettingen-research-online.de/dataset.xhtml?persistentId=doi:10.25625/P7VQTP&version=DRAFT and extract the ``pigtrack0001.mp4`` file from it. Then, navigate to ``MyDrive/PigDetect``, create a folder named ``example_videos`` there, and put ``pigtrack0001.mp4`` there. The full path in google drive to the video should be ``MyDrive/PigDetect/example_videos/pigtrack0001.mp4``.

The Co-DINO model is quite slow, especially here on Colab, so inference time for the 14 sec video is roughly 2 min. More lightweight models can easily be added to Colab. Feel free to write a github issue if you feel this would be helpful. Running inference on your custom videos can be done by simply replacing the example video with a video of your choice.

The model output is saved to ``MyDrive/PigDetect/example_videos_results``. It includes a visualization of the tracking results in mp4 format

In [ ]:
%cd ../../../tracking/boxmot

In [ ]:
!python main.py \
  --config configs/botsort.yaml \
  --inference_detector_checkpoint ../../../drive/MyDrive/PigDetect/codino_swin.pth \
  --seq_dir ../../../drive/MyDrive/PigDetect/example_videos \
  --outputs_base ../../../drive/MyDrive/PigDetect/example_videos_results